In [1]:
import os
import pandas as pd
import sqlite3

In [2]:
firefox_profile_dir = '/home/bird/.mozilla/firefox/iadzfbcv.default/'
[x for x in os.listdir(firefox_profile_dir) if x.endswith('sqlite')]

['cookies.sqlite',
 'storage-sync.sqlite',
 'kinto.sqlite',
 'storage.sqlite',
 'content-prefs.sqlite',
 'webappsstore.sqlite',
 'places.sqlite',
 'favicons.sqlite',
 'permissions.sqlite',
 'formhistory.sqlite']

In [3]:
storage_file = '{}/webappsstore.sqlite'.format(firefox_profile_dir)
storage_db = sqlite3.connect(storage_file)

In [4]:
# %load '/home/bird/Documents/tracking technologies/notebooks/get_sqlite_tables.py'
def list_tables_in_db(db):
    print(db.cursor().execute("SELECT name FROM sqlite_master WHERE type='table';").fetchall())

list_tables_in_db(storage_db)

[('webappsstore2',)]


In [5]:
storage_df = pd.read_sql('SELECT * FROM webappsstore2', storage_db)
storage_df.head()

,originAttributes,originKey,scope,key,value
0,,moc.ebutuoy.www.:https:443,moc.ebutuoy.www.:https:443,yt-remote-device-id,"{""data"":""57597ada-8b1f-4ff6-9ad5-10f9fddba032""..."
1,,moc.ollert.:https:443,moc.ollert.:https:443,sidebarState-527273a994b0712768004193,"{""idRecentBoards"":[""58fa22b3e5648003501dcf80""]}"
2,,moc.ollert.:https:443,moc.ollert.:https:443,ajs_user_traits,{}
3,,moc.ollert.:https:443,moc.ollert.:https:443,debug,undefined
4,,moc.ollert.:https:443,moc.ollert.:https:443,ajs_group_properties,{}


In [6]:
storage_df['origin'] = storage_df.originKey.apply(lambda x: x[::-1].split(':.')[1])
storage_df.head()

,originAttributes,originKey,scope,key,value,origin
0,,moc.ebutuoy.www.:https:443,moc.ebutuoy.www.:https:443,yt-remote-device-id,"{""data"":""57597ada-8b1f-4ff6-9ad5-10f9fddba032""...",www.youtube.com
1,,moc.ollert.:https:443,moc.ollert.:https:443,sidebarState-527273a994b0712768004193,"{""idRecentBoards"":[""58fa22b3e5648003501dcf80""]}",trello.com
2,,moc.ollert.:https:443,moc.ollert.:https:443,ajs_user_traits,{},trello.com
3,,moc.ollert.:https:443,moc.ollert.:https:443,debug,undefined,trello.com
4,,moc.ollert.:https:443,moc.ollert.:https:443,ajs_group_properties,{},trello.com


In [7]:
cookies_file = '{}/cookies.sqlite'.format(firefox_profile_dir)
cookies_db = sqlite3.connect(cookies_file)
cookied_df = pd.read_sql('SELECT * FROM moz_cookies', cookies_db)
shared_values = []
for v in cookied_df.value.unique():
    matches = cookied_df[cookied_df.value.str.contains(v, regex=False)]
    if len(matches.baseDomain.unique()) > 5:
        shared_values.append(v)
potential_ids = [x for x in shared_values if(len(x) > 10) & ('com' not in x)] 

In [8]:
# From cookie table
print('We have', len(potential_ids), 'potential ids. Things like:')
potential_ids[0:5]

We have 29 potential ids. Things like:


['7d3a5a8f-16c6-4a00-b672-f68cb04d1141',
 'AABkok61JQUAABIntsWgWg',
 'A798FAB6-8B6A-45E1-BDC9-9132ED3416A5',
 'GGc421K1ICVt',
 '8a88bb49-9ef4-42d6-a720-70cadad54101']

In [9]:
for potential_id in potential_ids:
    if len(storage_df[storage_df.value.str.contains(potential_id)]) > 0:
        print(potential_id, 'found in local storage')

67936421072632709762729202117726060613 found in local storage
7620423c-7103-4edc-9aee-099c75141b87-tuct18c03dc found in local storage


In [10]:
repeated_id = '67936421072632709762729202117726060613'
storage_df[storage_df.value.str.contains(repeated_id)][['value','origin']]

,value,origin
293,"{""email"":null,""timeIncId"":""53fdd635-5007-4f53-...",people.com


In [11]:
repeated_id = '7620423c-7103-4edc-9aee-099c75141b87-tuct18c03dc'
storage_df[storage_df.value.str.contains(repeated_id)][['value', 'origin']]

,value,origin
29,7620423c-7103-4edc-9aee-099c75141b87-tuct18c03dc,www.latimes.com
229,7620423c-7103-4edc-9aee-099c75141b87-tuct18c03dc,www.merriam-webster.com
231,v2_bf1ca6db22b70291779ce41eb2e5aee5_7620423c-7...,www.merriam-webster.com
234,v2_bf1ca6db22b70291779ce41eb2e5aee5_7620423c-7...,tpc.googlesyndication.com
238,7620423c-7103-4edc-9aee-099c75141b87-tuct18c03dc,tpc.googlesyndication.com
359,v2_9a6c86f52b3f9239067936a17472df9f_7620423c-7...,www.huffingtonpost.com
361,7620423c-7103-4edc-9aee-099c75141b87-tuct18c03dc,www.huffingtonpost.com
409,v2_66138b9f6bd0ec8912d19cb714efd912_7620423c-7...,www.latimes.com
988,7620423c-7103-4edc-9aee-099c75141b87-tuct18c03dc,www.bloomberg.com
993,v2_96ea2f0533c59e2312c1f1112ced8f46_7620423c-7...,www.bloomberg.com
